<a href="https://colab.research.google.com/github/HoseinNekouei/Building_shadow_detection/blob/main/Poisonous_Mushrooms_test_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!mkdir /content/data

In [11]:
!cp /content/drive/MyDrive/dataset/playground-series-s4e8_2.zip /content/data

In [12]:
!unzip /content/data/playground-series-s4e8_2.zip -d /content/data/

Archive:  /content/data/playground-series-s4e8_2.zip
  inflating: /content/data/sample_submission.csv  
  inflating: /content/data/test.csv  
  inflating: /content/data/train.csv  


In [20]:
import pandas as pd
import torch
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
import joblib  # For saving and loading the encoder

In [19]:
device= 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [56]:
df= pd.read_csv('/content/data/test.csv')
df

,id,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,3116945,8.64,x,NaN,n,t,NaN,NaN,w,11.13,...,b,NaN,w,u,w,t,g,NaN,d,a
1,3116946,6.90,o,t,o,f,NaN,c,y,1.27,...,NaN,NaN,n,NaN,NaN,f,f,NaN,d,a
2,3116947,2.00,b,g,n,f,NaN,c,n,6.18,...,NaN,NaN,n,NaN,NaN,f,f,NaN,d,s
3,3116948,3.47,x,t,n,f,s,c,n,4.98,...,NaN,NaN,w,NaN,n,t,z,NaN,d,u
4,3116949,6.17,x,h,y,f,p,NaN,y,6.73,...,NaN,NaN,y,NaN,y,t,NaN,NaN,d,u
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2077959,5194904,0.88,x,g,w,f,a,d,w,2.67,...,NaN,NaN,e,NaN,NaN,f,f,NaN,d,u
2077960,5194905,3.12,x,s,w,f,d,c,w,2.69,...,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a
2077961,5194906,5.73,x,e,e,f,a,NaN,w,6.16,...,NaN,NaN,y,NaN,w,t,z,NaN,d,a
2077962,5194907,5.03,b,g,n,f,a,d,g,6.00,...,NaN,s,g,NaN,NaN,f,f,NaN,d,a


In [57]:
df= df.drop(['id'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2077964 entries, 0 to 2077963
Data columns (total 20 columns):
 #   Column                Dtype  
---  ------                -----  
 0   cap-diameter          float64
 1   cap-shape             object 
 2   cap-surface           object 
 3   cap-color             object 
 4   does-bruise-or-bleed  object 
 5   gill-attachment       object 
 6   gill-spacing          object 
 7   gill-color            object 
 8   stem-height           float64
 9   stem-width            float64
 10  stem-root             object 
 11  stem-surface          object 
 12  stem-color            object 
 13  veil-type             object 
 14  veil-color            object 
 15  has-ring              object 
 16  ring-type             object 
 17  spore-print-color     object 
 18  habitat               object 
 19  season                object 
dtypes: float64(3), object(17)
memory usage: 317.1+ MB


In [58]:
df.isna().mean()

,0
cap-diameter,3.368682e-06
cap-shape,1.491845e-05
cap-surface,2.150682e-01
cap-color,6.256124e-06
does-bruise-or-bleed,4.812403e-06
gill-attachment,1.683480e-01
gill-spacing,4.040469e-01
gill-color,2.358077e-05
stem-height,4.812403e-07
stem-width,0.000000e+00


In [59]:
missing_threshold= 0.85

high_missing_value = df.columns[df.isna().mean() > missing_threshold]
high_missing_value

df= df.drop(columns=high_missing_value)
df

,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-surface,stem-color,has-ring,ring-type,habitat,season
0,8.64,x,NaN,n,t,NaN,NaN,w,11.13,17.12,NaN,w,t,g,d,a
1,6.90,o,t,o,f,NaN,c,y,1.27,10.75,NaN,n,f,f,d,a
2,2.00,b,g,n,f,NaN,c,n,6.18,3.14,NaN,n,f,f,d,s
3,3.47,x,t,n,f,s,c,n,4.98,8.51,NaN,w,t,z,d,u
4,6.17,x,h,y,f,p,NaN,y,6.73,13.70,NaN,y,t,NaN,d,u
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2077959,0.88,x,g,w,f,a,d,w,2.67,1.35,NaN,e,f,f,d,u
2077960,3.12,x,s,w,f,d,c,w,2.69,7.38,NaN,w,f,f,g,a
2077961,5.73,x,e,e,f,a,NaN,w,6.16,9.74,NaN,y,t,z,d,a
2077962,5.03,b,g,n,f,a,d,g,6.00,3.46,s,g,f,f,d,a


In [60]:
cat_col_name= df.select_dtypes(include=['object']).columns
print(cat_col_name)

num_col_name= df.select_dtypes(include=['float64']).columns
print(num_col_name)

Index(['cap-shape', 'cap-surface', 'cap-color', 'does-bruise-or-bleed',
       'gill-attachment', 'gill-spacing', 'gill-color', 'stem-surface',
       'stem-color', 'has-ring', 'ring-type', 'habitat', 'season'],
      dtype='object')
Index(['cap-diameter', 'stem-height', 'stem-width'], dtype='object')


In [61]:
mode_value= df[cat_col_name].mode()
mode_value= mode_value.loc[0].to_dict()
mode_value

{'cap-shape': 'x',
 'cap-surface': 't',
 'cap-color': 'n',
 'does-bruise-or-bleed': 'f',
 'gill-attachment': 'a',
 'gill-spacing': 'c',
 'gill-color': 'w',
 'stem-surface': 's',
 'stem-color': 'w',
 'has-ring': 'f',
 'ring-type': 'f',
 'habitat': 'd',
 'season': 'a'}

In [62]:
median_value= df[num_col_name].median(numeric_only= True)
median_value= median_value.to_dict()
median_value

{'cap-diameter': 5.74, 'stem-height': 5.88, 'stem-width': 9.64}

In [63]:
for column in df:
  if df[column].isna().any():
    if df[column].dtype=='object':
      df[cat_col_name] = df[cat_col_name].fillna(mode_value[column])
    else:
      df[num_col_name] = df[num_col_name].fillna(median_value[column])

df

,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-surface,stem-color,has-ring,ring-type,habitat,season
0,8.64,x,x,n,t,x,x,w,11.13,17.12,x,w,t,g,d,a
1,6.90,o,t,o,f,x,c,y,1.27,10.75,x,n,f,f,d,a
2,2.00,b,g,n,f,x,c,n,6.18,3.14,x,n,f,f,d,s
3,3.47,x,t,n,f,s,c,n,4.98,8.51,x,w,t,z,d,u
4,6.17,x,h,y,f,p,x,y,6.73,13.70,x,y,t,x,d,u
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2077959,0.88,x,g,w,f,a,d,w,2.67,1.35,x,e,f,f,d,u
2077960,3.12,x,s,w,f,d,c,w,2.69,7.38,x,w,f,f,g,a
2077961,5.73,x,e,e,f,a,x,w,6.16,9.74,x,y,t,z,d,a
2077962,5.03,b,g,n,f,a,d,g,6.00,3.46,s,g,f,f,d,a


In [64]:
df.isna().mean()

,0
cap-diameter,0.0
cap-shape,0.0
cap-surface,0.0
cap-color,0.0
does-bruise-or-bleed,0.0
gill-attachment,0.0
gill-spacing,0.0
gill-color,0.0
stem-height,0.0
stem-width,0.0


In [65]:
encoder_filename = 'ordinal_encoder.pkl'

# Load the encoder from the file
loaded_encoder = joblib.load(encoder_filename)

df[cat_col_name]= loaded_encoder.transform(df[cat_col_name].astype(str))
df[cat_col_name]

NotFittedError: This OrdinalEncoder instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [55]:
# Step 3: Load the scaler from the file
scaler_filename = 'standard_scaler.pkl'
loaded_scaler = joblib.load(scaler_filename)

scaler= StandardScaler()
df[num_col_name]= loaded_scaler.transform(df[num_col_name])
df[num_col_name]

NotFittedError: This StandardScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.